In [1]:
import numpy as np
from pydrake.multibody.tree import SpatialInertia, UnitInertia
from pydrake.multibody.plant import MultibodyPlant_, MultibodyPlant
from pydrake.autodiffutils import AutoDiffXd

In [2]:
plant = MultibodyPlant(time_step=0.0)

body1 = plant.AddRigidBody(
    "body1",
    M_BBo_B=SpatialInertia(
        mass=2.0,
        p_PScm_E=[0, 0, 0],
        # N.B. Rotational inertia is unimportant for calculations
        # in this notebook, and thus is arbitrarily chosen.
        G_SP_E=UnitInertia(0.1, 0.1, 0.1),
    ),
)
body2 = plant.AddRigidBody(
    "body2",
    M_BBo_B=SpatialInertia(
        mass=0.5,
        p_PScm_E=[0, 0, 0],
        # N.B. Rotational inertia is unimportant for calculations
        # in this notebook, and thus is arbitrarily chosen.
        G_SP_E=UnitInertia(0.1, 0.1, 0.1),
    ),
)
plant.Finalize()

In [3]:
# convert plant to AutoDiff compatible after Finalize()
plant_ad = plant.ToScalarType[AutoDiffXd]()
body1_ad = plant_ad.get_body(body1.index())
body2_ad = plant_ad.get_body(body2.index())
context_ad = plant_ad.CreateDefaultContext()

In [4]:
print(type(body1_ad))
print(body1_ad)

<class 'pydrake.multibody.tree.RigidBody_𝓣AutoDiffXd𝓤'>
<RigidBody_[AutoDiffXd] name='body1' index=1 model_instance=1>


In [5]:
# making m1 and m2 as dependent variables for AD
m1 = AutoDiffXd(2.0, [1.0, 0.0])
body1_ad.SetMass(context_ad, m1)
m2 = AutoDiffXd(0.5, [0.0, 1.0])
body2_ad.SetMass(context_ad, m2)

In this example, the gradients to be calculated using AD is the vertical force w.r.t each body's mass, i.e. $\frac{\partial \boldsymbol{f_z}}{\partial \boldsymbol{m}}$. Because this is a **MultibodyPlant()** with two rigid bodies so two masses, and two entries in $\boldsymbol{f}_z$, the gradient matrix $\frac{\partial \boldsymbol{f_z}}{\partial \boldsymbol{m}} = \begin{bmatrix} \frac{\partial f_{z1}}{m_1} &\frac{\partial f_{z2}}{m_1}\\ \frac{\partial f_{z1}}{m_2} &\frac{\partial f_{z2}}{m_2}\end{bmatrix} \in \mathbb{R}^{2\times2}$.
The 1st arg for AutoDiffXd() is the value of the mass. The 2nd arg specifies the where the dependent variables are, e.g. zero for the 2nd entry for $m_1$ because $f_{z1}$ is not dependent on m2. Similarly for $m_2$.

In [6]:
def get_z_component(plant, body, v):
    assert body.is_floating()
    
    # body.floating_velocities_start() returns the index of this Body’s first generalized 
    # velocity in the full state vector for a MultibodyPlant model, under the dubious 
    # assumption that the state consists of [q v] concatenated.
    x_start = body.floating_velocities_start()

    # floating velocities conventions [angular; translation], so spatial velocity convention in Plücker Notation
    v_start = x_start - plant.num_positions() # get the v starting index
    nv_pose = 6

    # damn! a detour just to get the translational force in gravity wrench and you use "v" as input arguments for confusion.
    # For those who are not familiar in spatial notation or "Plücker" notation. twist (spatial velocity) v = [angular linear]^T 
    # and wrench (spatial force) f = [moment force]^T. The indexing for angular and linear are the same.     
    """
    Actually, it is such a good example that it illustrated how the state vector is arranged (though still conform to [q v])
    for MultibodyPlant():
    state = [q1, q2, ..., qN, dq1, dq2, ..., dqN], that is why nv_pose = 6 here.
    """
    rxyz_txyz = v[v_start:v_start + nv_pose]
    assert len(rxyz_txyz) == nv_pose 
    txyz = rxyz_txyz[-3:] # get the translation part
    z = txyz[2] # [x y z]
    return z

In [7]:
@np.vectorize
def ad_to_string(x):
    # Formats an array of AutoDiffXd elements to a string.
    # Note that this implementation is for a scalar, but we use `np.vectorize` to
    # effectively convert our array to `ndarray` of strings
    return f"AutoDiffXd({x.value()}, derivatives={x.derivatives()})"

In [8]:
tau_g = plant_ad.CalcGravityGeneralizedForces(context_ad)
tau_g_z1 = get_z_component(plant_ad, body1_ad, tau_g)
tau_g_z2 = get_z_component(plant_ad, body2_ad, tau_g)
print(ad_to_string(tau_g_z1))
print(ad_to_string(tau_g_z2))

AutoDiffXd(-19.62, derivatives=[-9.81 -0.  ])
AutoDiffXd(-4.905, derivatives=[-0.   -9.81])


In [9]:
tau_g

array([<AutoDiffXd -0.0 nderiv=2>, <AutoDiffXd -0.0 nderiv=2>,
       <AutoDiffXd -0.0 nderiv=2>, <AutoDiffXd -0.0 nderiv=2>,
       <AutoDiffXd -0.0 nderiv=2>, <AutoDiffXd -19.62 nderiv=2>,
       <AutoDiffXd -0.0 nderiv=2>, <AutoDiffXd -0.0 nderiv=2>,
       <AutoDiffXd -0.0 nderiv=2>, <AutoDiffXd -0.0 nderiv=2>,
       <AutoDiffXd -0.0 nderiv=2>, <AutoDiffXd -4.905 nderiv=2>],
      dtype=object)

In [34]:
tau_g_z1

<AutoDiffXd -19.62 nderiv=2>

In [29]:
tau_g_z1.derivatives()

array([-9.81, -0.  ])

In [30]:
tau_g_z1.value()

-19.62

In [10]:
type(tau_g_z1)

pydrake.autodiffutils.AutoDiffXd

In [ ]:
from pydrake.all import ExtractGradient
ExtractGradient(tau_g_z1